In [3]:
import requests
from bs4 import BeautifulSoup as BS
import csv
import re

Get the sitemap from National Trust

In [4]:
nturl = "https://www.nationaltrust.org.uk/sitemap-place.xml"

nt_page = requests.get(nturl).text

Read it through beautiful soup as XML

In [5]:
soup = BS(nt_page, "xml")

Find all the links and get just the href

In [6]:
data_list = soup.find_all("xhtml:link")

In [7]:
alinks = [i.get("href") for i in data_list]

# remove duplicates
dedupe = []
for x in alinks:
    if x not in dedupe:
        dedupe.append(x)

# set up data structure to store values
NatTrust = {
    "property1" : [],
    "section" : [],
    "ticket" : [],
    "type" : [],
    "price" : [],
    "lat":[],
    "lon":[],
}
AdFree = []

In [10]:
for i in dedupe[:10]:
#clean the URL and add prices to the end
    placeclean = ('/').join(i.split('/')[0:4])
    placeurl = placeclean +"#Prices"
#the bits for the lat long
    directionsurl=placeclean+"#How to get here"
    directions_page = requests.get(directionsurl).text
    directions_soup=BS(directions_page,"html.parser")
    directionlink=directions_soup.find("a",{"id":"nt.directions.google-link"})
    if directionlink:
        latlon = directionlink['href'].split('//')[2]
        lat = latlon.split(',')[0]
        lon = latlon.split(',')[1]
    else:
        continue
    print(placeurl)
#request the URL and put it through BS 
    place_page = requests.get(placeurl).text
    place_soup=BS(place_page,"html.parser")
#Get the title
    title = place_soup.find("meta",  property="og:title")
    if title:
        property1 = title["content"]
        print(property1)
    else:
        continue
#get the table of prices    
    prices = place_soup.find("div",{"id":"nt.prices"})
    if prices:
        section = prices.find_all("div",{"class":"section"})
        if section:
            for i in section:
                sectiontitle = i.find("span")
                category = sectiontitle.text
                pricesTable = i.find("table").find("tbody").find_all("tr")
                for i in pricesTable:
                    td = i.find_all("td")
                    for index,i in enumerate(td):
                        if index==0:
                            span=i.find("span")
                            if span:
                                ticket=span.text
                            else:
                                ticket=i.text
                        elif index==1:
                            gaprice=re.sub(r'\s+', '', i.text)
                            NatTrust["property1"].append(property1)
                            NatTrust["section"].append(category)
                            NatTrust["ticket"].append(ticket)
                            NatTrust["type"].append("Gift aid")
                            NatTrust["price"].append(gaprice)
                            NatTrust["lat"].append(lat)
                            NatTrust["lon"].append(lon)
                        elif index==2:
                            standard = re.sub(r'\s+', '', i.text)
                            NatTrust["property1"].append(property1)
                            NatTrust["section"].append(category)
                            NatTrust["ticket"].append(ticket)
                            NatTrust["type"].append("Standard")
                            NatTrust["price"].append(standard)
                            NatTrust["lat"].append(lat)
                            NatTrust["lon"].append(lon)
        else:
            free = prices.find("h2",{"id":"nt.prices.free.admission.notice"})
#             print(property, free.text)
            AdFree.append(property)
    else:
        continue
        
print(NatTrust)
print(AdFree)

https://www.nationaltrust.org.uk/segontium#Prices
Segontium
https://www.nationaltrust.org.uk/clyston-mill#Prices
Clyston Mill
https://www.nationaltrust.org.uk/mendip-hills#Prices
Mendip Hills
https://www.nationaltrust.org.uk/ulverscroft-nature-reserve#Prices
Ulverscroft Nature Reserve
https://www.nationaltrust.org.uk/white-cliffs-dover#Prices
The White Cliffs of Dover
https://www.nationaltrust.org.uk/bransdale#Prices
Bransdale
https://www.nationaltrust.org.uk/allen-banks-and-staward-gorge#Prices
Allen Banks and Staward Gorge
https://www.nationaltrust.org.uk/duffield-castle#Prices
Duffield Castle
https://www.nationaltrust.org.uk/sutton-house#Prices
Sutton House and Breaker's Yard
{'property1': ['Clyston Mill', 'Clyston Mill', 'Clyston Mill', 'Clyston Mill', 'Clyston Mill', 'Clyston Mill', 'Clyston Mill', 'Clyston Mill', 'The White Cliffs of Dover', 'The White Cliffs of Dover', 'The White Cliffs of Dover', 'The White Cliffs of Dover', 'The White Cliffs of Dover', 'The White Cliffs of Dov

In [11]:
with open("NatTrust.csv", "w", encoding="utf-8") as NatTrustCSV:
    fields = ["property1","section","ticket","type","price","lat","lon"]
    writer = csv.DictWriter(NatTrustCSV, fields)
    writer.writeheader()
    for index,i in enumerate(NatTrust["property1"]):
        row={"property1":NatTrust["property1"][index],"section":NatTrust["section"][index],"ticket":NatTrust["ticket"][index],"type":NatTrust["type"][index],"price":re.sub(r'\s+', '',NatTrust["price"][index]),"lat":NatTrust["lat"][index],"lon":NatTrust["lon"][index]}
        writer.writerow(row)

In [12]:
print(NatTrust["price"])

['N/A', '£5.80', 'N/A', '£2.90', 'N/A', '£14.50', 'N/A', '£8.70', 'N/A', '£4.00', 'N/A', '£10.00', 'N/A', '£5.00', '£7.00', '£6.30', '£3.50', '£3.00', '£17.50', '£15.50', '£10.50', '£9.50', 'N/A', '£5.40', 'N/A', '£5.80', 'N/A', '£2.90', 'N/A', '£14.50', 'N/A', '£8.70', 'N/A', '£4.00', 'N/A', '£10.00', 'N/A', '£5.00', '£7.00', '£6.30', '£3.50', '£3.00', '£17.50', '£15.50', '£10.50', '£9.50', 'N/A', '£5.40']
